In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent.parent))

from tasks.simple_MCQA.simple_MCQA import get_token_positions, get_counterfactual_datasets, get_causal_model
from experiments.aggregate_experiments import residual_stream_baselines
from neural.pipeline import LMPipeline
from experiments.filter_experiment import FilterExperiment
import gc
import torch
import os

gc.collect()
torch.cuda.empty_cache()

# Get counterfactual datasets and causal model
counterfactual_datasets = get_counterfactual_datasets(hf=True, size=None)
causal_model = get_causal_model()

# Print available datasets
print("Available datasets:", counterfactual_datasets.keys())

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def clear_memory():
    # Clear Python garbage collector
    gc.collect()
    
    # Clear CUDA cache if available
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    # Force a synchronization point to ensure memory is freed
    if torch.cuda.is_available():
        torch.cuda.synchronize()


def checker(output_text, expected):
    return expected in output_text

model_name = "google/gemma-2-2b"
pipeline = LMPipeline(model_name, max_new_tokens=1, device=device, dtype=torch.float16)
pipeline.tokenizer.padding_side = "left"
batch_size = 32
print("DEVICE:", pipeline.model.device)

# Get a sample input and check model's prediction
sampled_example = next(iter(counterfactual_datasets.values()))[0]
print("INPUT:", sampled_example["input"])
print("EXPECTED OUTPUT:", causal_model.run_forward(sampled_example["input"])["raw_output"])
print("MODEL PREDICTION:", pipeline.dump(pipeline.generate(sampled_example["input"])))

# Filter the datasets based on model performance
print("\nFiltering datasets based on model performance...")
exp = FilterExperiment(pipeline, causal_model, checker)
filtered_datasets = exp.filter(counterfactual_datasets, verbose=True, batch_size=1024)

token_positions = get_token_positions(pipeline, causal_model)

# Display token highlighting for a sample
print("\nToken positions highlighted in samples:")
for dataset in filtered_datasets.values():
    for token_position in token_positions:
        example = dataset[0]
        print(token_position.highlight_selected_token(example["counterfactual_inputs"][0]))
        break
    break

gc.collect()
torch.cuda.empty_cache()

start = 0 
end = 1

# Use original config for all models
config = {"batch_size": 64, "evaluation_batch_size": 1024, "training_epoch": 1, "n_features": 16, "regularization_coefficient": 0.0, "output_scores": False}
    
names = ["answerPosition", "randomLetter", "answerPosition_randomLetter"]

# Prepare train and test data dictionaries
train_data = {}
test_data = {}

for name in names:
    if name + "_train" in filtered_datasets:
        train_data[name + "_train"] = filtered_datasets[name + "_train"]
    if name + "_test" in filtered_datasets:
        test_data[name + "_test"] = filtered_datasets[name + "_test"]
    # Uncomment the line below if testprivate datasets are available
    # if name + "_testprivate" in filtered_datasets:
    #     test_data[name + "_testprivate"] = filtered_datasets[name + "_testprivate"]

verbose = False 
results_dir = "mock_submission_results"
model_dir = os.path.join("mock_submission", "4_answer_MCQA_Gemma2ForCausalLM_answer_pointer")

nnsight is not detected. Please install via 'pip install nnsight' for nnsight backend.


Using the latest cached version of the dataset since mib-bench/copycolors_mcqa_private_test couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration '4_answer_choices' at /home/atticus/.cache/huggingface/datasets/mib-bench___copycolors_mcqa_private_test/4_answer_choices/0.0.0/da600e08a8c9fe40917ac887eda693de57b9f04d (last modified on Tue Jun  3 19:36:15 2025).


Available datasets: dict_keys(['answerPosition_train', 'randomLetter_train', 'answerPosition_randomLetter_train', 'answerPosition_validation', 'randomLetter_validation', 'answerPosition_randomLetter_validation', 'answerPosition_test', 'randomLetter_test', 'answerPosition_randomLetter_test', 'answerPosition_testprivate', 'randomLetter_testprivate', 'answerPosition_randomLetter_testprivate'])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You have set `use_cache` to `False`, but cache_implementation is set to hybrid. cache_implementation will have no effect.


...

Dataset 'answerPosition_testprivate': kept 50/50 examples (100.0%)


Filtering randomLetter_testprivate: 100%|██████████| 1/1 [00:00<00:00,  1.45it/s]


Dataset 'randomLetter_testprivate': kept 34/50 examples (68.0%)


Filtering answerPosition_randomLetter_testprivate: 100%|██████████| 1/1 [00:00<00:00,  1.21it/s]

Dataset 'answerPosition_randomLetter_testprivate': kept 33/50 examples (66.0%)

Total filtering results:
Original examples: 780
Kept examples: 612
Overall keep rate: 78.5%

Token positions highlighted in samples:
<bos>Question: Coconuts are brown. What color are coconuts?
A. red
B. orange
C. purple
**D**. brown
Answer:


In [3]:
import os

target_variables=["answer_pointer"]

In [4]:
# Run DAS method (Direct Attribution with Subspace)
print("Running DAS method...")

residual_stream_baselines(
    pipeline=pipeline,
    task=causal_model,
    token_positions=token_positions,
    train_data=train_data,
    test_data=test_data,
    config=config,
    target_variables=target_variables,
    checker=checker,
    start=start,
    end=end,
    verbose=verbose,
    model_dir=model_dir,
    results_dir=results_dir,
    methods=["DAS"]  # Only run DAS method
)

clear_memory()

Running DAS method...
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0


Epoch: 100%|██████████| 1/1 [00:03<00:00,  3.50s/it]


Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0


Epoch: 100%|██████████| 1/1 [00:03<00:00,  3.03s/it]


Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0


Epoch: 100%|██████████| 1/1 [00:02<00:00,  2.84s/it]


Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0


In [ ]:
# Example: Run DBM+SAE method (uncomment to use)
# NOTE: This method requires the sae_lens library and is specific to certain models

# print("Running DBM+SAE method...")

# residual_stream_baselines(
#     pipeline=pipeline,
#     task=causal_model,
#     token_positions=token_positions,
#     train_data=train_data,
#     test_data=test_data,
#     config=config,
#     target_variables=target_variables,
#     checker=checker,
#     start=start,
#     end=end,
#     verbose=verbose,
#     model_dir=model_dir,
#     results_dir=results_dir,
#     methods=["DBM+SAE"]  # Only run DBM+SAE method
# )

# clear_memory()

In [ ]:
# Example: Run DBM method (uncomment to use)
# print("Running DBM method...")

# residual_stream_baselines(
#     pipeline=pipeline,
#     task=causal_model,
#     token_positions=token_positions,
#     train_data=train_data,
#     test_data=test_data,
#     config=config,
#     target_variables=target_variables,
#     checker=checker,
#     start=start,
#     end=end,
#     verbose=verbose,
#     model_dir=model_dir,
#     results_dir=results_dir,
#     methods=["DBM"]  # Only run DBM method
# )

# clear_memory()

In [5]:
# Example: Load saved models and run inference
# This demonstrates how to load previously trained featurizers and run interventions

from experiments.residual_stream_experiment import PatchResidualStream

print("Loading trained models and running inference...")

for method in ["DAS"]:  # Can also test "DBM", "DBM+SAE", etc.
    print(f"Testing {method} method...")
    
    # Create experiment with same configuration
    config["method_name"] = method
    experiment = PatchResidualStream(pipeline, causal_model, list(range(start,end)), token_positions, checker, config=config)
    
    # Set up SAE loader if needed for DBM+SAE method
    if method == "DBM+SAE":
        from sae_lens import SAE
        def sae_loader(layer):
            sae, _, _ = SAE.from_pretrained(
                release = "gemma-scope-2b-pt-res-canonical",
                sae_id = f"layer_{layer}/width_16k/canonical",
                device = "cpu",
            )
            return sae
        experiment.build_SAE_feature_intervention(sae_loader)
    
    # Load the trained featurizers
    method_model_dir = os.path.join(model_dir, method) if method != "DAS" else model_dir
    experiment.load_featurizers(method_model_dir)
    
    # Run interventions on test data
    raw_results = experiment.perform_interventions(test_data, verbose=verbose, target_variables_list=[target_variables], save_dir=results_dir + "_loaded")
    
    # Clean up
    del experiment, raw_results
    clear_memory()

print("Inference completed!")

Loading trained models and running inference...
Testing DAS method...
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Intervention key: layer_0_comp_block_output_unit_pos_nunit_1#0
Inference completed!
